<a href="https://colab.research.google.com/github/DevJGraham/vlm-image-to-description-generator/blob/main/Google_Vision_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
# So that the output is not captured for Github
%%capture --no-stderr

cd drive/MyDrive/CS/WGU/Computer\ Science\ Capstone\ -\ C964

# Google Vision API

In [4]:
!pip install -U google-cloud-vision google-auth google-auth-oauthlib -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.3 which is incompatible.


In [5]:
# Sign in to Google so the Cloud SDK in this VM can act as your user
from google.colab import auth
auth.authenticate_user()

In [6]:
# Point the CLI at the correct project
!gcloud config set project ocr-test-graham

Updated property [core/project].


In [21]:
# Log in to my Google account and get back an authorization code which then the CLI exchanges for a short-term access token, and a log-term refresh token that can refressh the access token when it expires
!gcloud auth application-default login

In [22]:
# So that the output is not captured for Github
%%capture --no-stderr

# Attach a quota/billing project to those user ADC credentials
!gcloud auth application-default set-quota-project ocr-test-graham

# Enable Vision API for the active CLI project
!gcloud services enable vision.googleapis.com

In [9]:
# Ensures the primary environment variable is unset so that the CLI will use the ADC instead of the primary env
# This ensures Python libs will actually use ADC (not a hard-coded service account key)

import os
print("GOOGLE_APPLICATION_CREDENTIALS set?", "GOOGLE_APPLICATION_CREDENTIALS" in os.environ)
os.environ.pop("GOOGLE_APPLICATION_CREDENTIALS", None)

GOOGLE_APPLICATION_CREDENTIALS set? False


In [23]:
# So that the output is not captured for Github
%%capture --no-stderr

# Prints the active project and the account for the CLI to verify that the previous steps worked
!gcloud config list

In [24]:
# So that the output is not captured for Github
%%capture --no-stderr

import google.auth

# Fetches the default credentials and the project that the client will use
creds, proj = google.auth.default()
print("ADC project seen by client:", proj) # proj should be the project that we are working with

In [12]:
from google.cloud import vision

In [13]:
client = vision.ImageAnnotatorClient()

# Read image as a byte file
with open("Images/ryobi.png", "rb") as f:
    content = f.read()

# Pass in a byte file image to Vision.Image creating a client image on which the vision API tasks can be performed
image = vision.Image(content=content)

text = client.text_detection(image=image) # Extracts text from the image
label = client.label_detection(image=image) # labels the items in the image
print("Text error message:", text.error.message)
print("Label error message:", label.error.message)

Text error message: 
Label error message: 


In [14]:
# Get all of the individual texts from the text_annotations
texts = []
for t in text.text_annotations[1:]:
  if t.description not in texts:
    texts.append(t.description)

texts

['RYOBL', 'RYOBI', 'RYOJ', '45']

In [15]:
# Get all of the individual labels from the label_annotations
labels = []
for l in label.label_annotations:
  labels.append(l.description)

labels

['Machine',
 'Saw',
 'Power tool',
 'Miter saw',
 'Tool',
 'Mitre saws',
 'Circular saw',
 'Plastic',
 'Abrasive saw',
 'Blade']

# Run Cloud Vision API On Each Image
> Take each image from GCS, extract text and vision for each image, return a list of dictionaries with this format

| Auction | Item Number | Image Number | Text | Labels |
|---------|-------------|--------------|------|--------|
| 'Aypt2025-31' | 1 | 1 | ['Ryobi', 'Model NO: 1234', ...] | ['Tool', 'Drill', ... ] |

In [16]:
from google.cloud import storage

In [17]:
storage_client = storage.Client()

In [4]:
# # This is needed to access the Google Cloud Storage bucket
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
bucket_name = "auction-images-bucket"

# Get bucket reference
bucket = storage_client.bucket(bucket_name)

# List all blobs (files) in the bucket
item_list = []
for blob in bucket.list_blobs():
    if blob.name.endswith('.jpg'):
      print(f"Processing: {blob.name}")
      # byte_data = blob.download_as_bytes()

In [ ]:
def format_dict():


In [89]:
def vision_cloud_api_call(image_bytes):
  # Pass in a byte file image to Vision.Image creating a client image on which the vision API tasks can be performed
  image = vision.Image(content=image_bytes)

  extracted_text = client.text_detection(image=image) # Extracts text from the image
  extracted_labels = client.label_detection(image=image) # labels the items in the image
  if text.error.message:
      print("Text error message:", text.error.message)
  if label.error.message:
    print("Label error message:", label.error.message)

In [85]:
def get_texts(vision_text_output):
  # Get all of the individual texts from the text_annotations
  texts = []
  for t in vision_text_output.text_annotations[1:]:
    if t.description not in texts:
      texts.append(t.description)

  return texts

def get_labels(vision_label_output):
  # Get all of the individual labels from the label_annotations
  labels = []
  for l in vision_label_output.label_annotations:
    labels.append(l.description)

  return labels

In [60]:
for blob in bucket.list_blobs():
  if blob.name == 'aypt2025-31/10_1.jpg':
    test_bytes = blob.download_as_bytes()

In [86]:
get_labels(example_label)

[]

In [84]:
get_texts(example_text)

['ARMSTRONG',
 'RUT',
 'GUARANTEE',
 'TOOLS',
 'ACTIN',
 'et',
 'th',
 'AIR',
 'TORK',
 'ARE',
 'B',
 'THE',
 'USA',
 'FETIME',
 'GUARANT',
 'MOLEC',
 'PA',
 'E',
 'woww',
 'S',
 'OOLS',
 'ARMSTRING',
 'MO']

In [52]:
item_list[0]

'aypt2025-31/10_1.jpg'

In [54]:
from PIL import Image

In [55]:
Image.open('aypt2025-31/10_1.jpg')

FileNotFoundError: [Errno 2] No such file or directory: 'aypt2025-31/10_1.jpg'

In [ ]:
auction = 'aypt2025-31/'
client = vision.ImageAnnotatorClient()

def cloud_vision_text_labels(image_path, auction):
  # Read image as a byte file
  with open(image_path, "rb") as f:
      content = f.read()

  # Pass in a byte file image to Vision.Image creating a client image on which the vision API tasks can be performed
  image = vision.Image(content=content)

  text = client.text_detection(image=image) # Extracts text from the image
  label = client.label_detection(image=image) # labels the items in the image
  print("Text error message:", text.error.message)
  print("Label error message:", label.error.message)
  return text, label

In [10]:
test = 'aypt2025-31/10_1.jpg'

In [15]:
auction = test[:test.find('/')]
image_num = test[test.find('/') + 1:test.find('.jpg')]

In [16]:
print(auction)
print(image_num)

aypt2025-31
10_1


In [30]:
test_list = [{'auction': auction, 'image_num': image_num, 'text': ['test_text1'], 'labels': ['test_labels1']},
{'auction': 'aypt2025-31', 'image_num': '10_2', 'text': ['test_text2'], 'labels': ['test_labels2']},
{'auction': 'aypt2025-31', 'image_num': '10_3', 'text': ['test_text3'], 'labels': ['test_labels3']}]

In [31]:
test_list

[{'auction': 'aypt2025-31',
  'image_num': '10_1',
  'text': ['test_text1'],
  'labels': ['test_labels1']},
 {'auction': 'aypt2025-31',
  'image_num': '10_2',
  'text': ['test_text2'],
  'labels': ['test_labels2']},
 {'auction': 'aypt2025-31',
  'image_num': '10_3',
  'text': ['test_text3'],
  'labels': ['test_labels3']}]

In [29]:
concatenated_list = []
for img in test_list:
  if img['image_num'][:img['image_num'].find('_')] == '10':


10
10_1
10
10_2
10
10_3
